In [ ]:
## Strategy WalkThrough  [EMA/Stoch RSI/ATR]

## First we need few indicators for buy and sell signal

1. EMA8 , EMA14 and EMA50  [EMA = Exponential Moving Average]

2. Stochastic RSI%k , Stochastic RSI%k 

  **The StochRSI oscillator was developed to take advantage of both momentum indicators in order to create a
    more sensitive indicator that is attuned to a specific security’s historical performance rather than a
    generalized analysis of price change.
    
3. ATR [ATR = Average True Range]
  
  **The indicator provide an indication of the degree of price volatility. Strong moves, in either direction,
    are often accompanied by large ranges, or large True Ranges.

# Buying condition

The close price  should be  above  EMA8 and EMA8 > EMA14 and EMA14 > EMA50
The sochrsi_k > sochrsi_d
Where EMA means Exponential Moving Average of different time frame


# Selling condition

After getting a buying signal , we'll find next close date where close >= current TotalProfit or < StockLoss . If any of the 
condition is satisfied we'll mark it for sell.



In [ ]:
!pip install yfinance

In [ ]:
# Importing Neccessary libraries

import yfinance as yf  # for loading data from online
import pandas as pd
import numpy as np
import ta              # for technical indicators


In [ ]:
def checkcross(df):
    
    """In this function wer're only checking if k value crosses d value at current row,
       but not in the previous row."""
    
    series = df['stochrsi_k'] > df['stochrsi_d']
    return series.diff()

def get_sell_date_outcome(df):
    
    """This function will find selling point and the outcome of the sell, wether it is Profit or 
       Stock Loss."""
    
    # If we reach Target Profit or Stock Loss from the Buying Signal, that's were we will sell

    selldates = []
    outcome = []

    for i in range(len(df)):     # Iterating throuh the dataframe
        
        """If a buying signal found , then we'll start another loop until the conditions are met."""
        
        if df.Buysignal.iloc[i]:
            k = 1
            SL = df.SL.iloc[i]
            TP = df.TP.iloc[i]
            in_position = True

            while in_position:
                looping_close = df.Close.iloc[i + k] # i + k closing price  

                if looping_close >= TP:   # Condition for Total Profit
                    selldates.append(df.iloc[i + k].name)
                    outcome.append('TP')
                    in_position = False

                elif looping_close <= SL:  # Condition for Stock Loss
                    selldates.append(df.iloc[i + k].name)
                    outcome.append('SL')
                    in_position = False

                k += 1
    return selldates, outcome


In [ ]:
def strategy(df):
    
    # Calculating [Stochastic RSI k/d] and inserting them as columns         
    
    df['stochrsi_k'] = ta.momentum.stochrsi_k(df.Close)
    df['stochrsi_d'] = ta.momentum.stochrsi_d(df.Close)
    
    # Inserting Exponential Average for windows = [8, 14 and 50]
    
    for i in (8,14,50):
        df['EMA_'+str(i)] = ta.trend.ema_indicator(df.Close,window=i)
        
    # ATR -> Average True Range
    df['atr'] = ta.volatility.average_true_range(df.High,df.Low,df.Close)
    
    # Droping Nan values
    
    df.dropna(inplace=True)
    
    # Checking for cross
    df['cross'] = checkcross(df)
    
    # Again droping null values
    df.dropna(inplace=True)
    
    # Adding current stock_loss and total_profit to calculate buy and sell 
    
    df['TP'] = df.Close + (df.atr * 2)
    df['SL'] = df.Close - (df.atr * 3)
    

    
    # Getting buy signal
    df['Buysignal'] = np.where((df.cross) & (df.Close > df.EMA_8) &
                          (df.EMA_8 > df.EMA_14) & (df.EMA_14 > df.EMA_50), 1, 0)
    
    # Getting sell signal and it's outcome , eg: Stock Loss or Target Profit
    sell,outcome = get_sell_date_outcome(df)
    
    # Adding sell & outcome signal to dataframe
    df.loc[sell, 'Sellsignal'] = 1
    df.loc[sell, 'Outcome'] = outcome
    
    # Filling up Nan values of sell signal to filter out unneccessary columns
    df.Sellsignal = df.Sellsignal.fillna(0).astype(int)
    df = df[(df.Buysignal==1)|(df.Sellsignal==1)]
    df = df[(df.Buysignal.diff()==1) | (df.Sellsignal.diff()==1)]
    
    return df
    

In [ ]:
# Optional live dataset from online

df = yf.download('EURUSD=x', start='2021-01-01', interval='1h')
data = strategy(df).head(10)

In [ ]:
import os
path = 'train/daily/' # change this variable if required
files = [i for i in os.listdir(path) if '.csv' in i]

results = []
original_df = []

for file in files:
    print(file)
    df = pd.read_csv(path+file, index_col=0) # Reading csv and setting Date column as index column
    copy = df
    df.index = pd.to_datetime(df.index) # changing data type of Date column
    df.rename(columns={'open': 'Open', 'close':'Close','high':'High','low':'Low','volume':'Volume'}, inplace=True)
    
    try:
        results.append(strategy(df))
    except:
        pass
    
    # for visualization purpose only
    copy = df
    for i in (8,14,50):
        copy['EMA_'+str(i)] = ta.trend.ema_indicator(copy.Close,window=i)
    copy['stochrsi_k'] = ta.momentum.stochrsi_k(copy.Close)
    copy['stochrsi_d'] = ta.momentum.stochrsi_d(copy.Close)
    original_df.append(copy)
    

In [ ]:
# plotting EMAs againest closing price for first data set : "ACG.csv"
import matplotlib.pyplot as plt

plt.figure(figsize=(15,5))
plt.plot(original_df[0].Close, label='Closing price')
plt.plot(original_df[0].EMA_8, c='green', alpha=0.7,label='EMA 8')
plt.plot(original_df[0].EMA_14, c='orange', alpha=0.7, label='EMA 14')
plt.plot(original_df[0].EMA_50, c='red', alpha=0.7, label='EMA 50')
plt.legend()
plt.show()

In [ ]:
# Plotting Stoach k/d 
# Stoch line is much more accurate then other indicators.

plt.figure(figsize=(17,2))
plt.plot(original_df[0].stochrsi_k, label='stochrsi_k line')
plt.plot(original_df[0].stochrsi_d, c='red', label='stochrsi_d line')
plt.show()

In [ ]:
# In case you're figuring out how will you evaluate matrices , you can simply access the final datasets by calling "results"
# variable. More specifically results[i] variable

results[0] # results[i] using a loop 

# Buying signal indicats when to buy , and selling signal indicates when to sell. After selling there is another 
# Indicator outcome which tells if the trade was profitable or loss. 
# Incase you're wondoring , TP -> Total Profit , SL -> Stock Loss, NaN -> Do nothing.

